<a href="https://colab.research.google.com/github/tushargupta123/Machine-learning/blob/main/XGBOOST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.2 MB/s eta 0:00:00


In [2]:
import optuna
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [3]:
df=pd.read_csv("https://raw.githubusercontent.com/sunnysavita10/svm_kernel/main/Admission_Prediction.csv")

In [4]:
df.head()


,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,NaN,104.0,3.0,3.0,3.5,8.00,1,0.72
3,4,322.0,110.0,3.0,3.5,2.5,8.67,1,0.80
4,5,314.0,103.0,2.0,2.0,3.0,8.21,0,0.65


In [5]:
df.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,500.000000,485.000000,490.000000,485.000000,500.000000,500.00000,500.000000,500.000000,500.00000
mean,250.500000,316.558763,107.187755,3.121649,3.374000,3.48400,8.576440,0.560000,0.72174
std,144.481833,11.274704,6.112899,1.146160,0.991004,0.92545,0.604813,0.496884,0.14114
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.00000,6.800000,0.000000,0.34000
25%,125.750000,308.000000,103.000000,2.000000,2.500000,3.00000,8.127500,0.000000,0.63000
50%,250.500000,317.000000,107.000000,3.000000,3.500000,3.50000,8.560000,1.000000,0.72000
75%,375.250000,325.000000,112.000000,4.000000,4.000000,4.00000,9.040000,1.000000,0.82000
max,500.000000,340.000000,120.000000,5.000000,5.000000,5.00000,9.920000,1.000000,0.97000


In [6]:
df.isnull().sum()

Serial No.            0
GRE Score            15
TOEFL Score          10
University Rating    15
SOP                   0
LOR                   0
CGPA                  0
Research              0
Chance of Admit       0
dtype: int64

In [7]:
df["GRE Score"]=df["GRE Score"].fillna(df["GRE Score"].median())
df["TOEFL Score"]=df["TOEFL Score"].fillna(df["TOEFL Score"].median())
df["University Rating"]=df["University Rating"].fillna(df["University Rating"].median())

In [8]:
X=df.drop(["Serial No.","Chance of Admit"],axis=1)

In [9]:
y=df[['Chance of Admit']]

In [10]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=20)

In [11]:
std_sca=StandardScaler()

In [12]:
transformed_x_train = std_sca.fit_transform(X_train)

In [13]:
transformed_x_test = std_sca.transform(X_test)

In [14]:
def objective(trial, data=X, target=y):
    train_x, test_x, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=25)
    param = {
        "tree_method": "gpu_hist",
        "lambda": trial.suggest_float('lambda', 1e-4, 10.0),
        "alpha": trial.suggest_float('alpha', 1e-4, 10.0),
        "colsample_bytree": trial.suggest_categorical('colsample_bytree', [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]),
        "subsample": trial.suggest_categorical('subsample', [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]),
        "learning_rate": trial.suggest_categorical('learning_rate', [0.00001, 0.0003, 0.008, 0.02, 0.01, 1, 8]),
        "n_estimators": 3000,
        "max_depth": trial.suggest_categorical('max_depth', [3, 4, 5, 6, 7, 8, 9, 10, 11, 12]),
        "random_state": trial.suggest_categorical("random_state", [10, 20, 30, 2000, 3454, 22323]),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 200)
    }
    Xgb_reg_model = xgb.XGBRegressor(**param)
    Xgb_reg_model.fit(train_x, y_train)
    y_pred = Xgb_reg_model.predict(test_x)
    mse = mean_squared_error(y_test, y_pred)
    return mse

In [15]:
find_param = optuna.create_study()
find_param.optimize(objective, n_trials=10)

[I 2023-09-06 05:40:31,166] A new study created in memory with name: no-name-ade1c438-1b40-405c-a8e0-5baf314bf50f
[I 2023-09-06 05:40:34,382] Trial 0 finished with value: 0.008504432668280541 and parameters: {'lambda': 0.8174218026257434, 'alpha': 9.82841996752315, 'colsample_bytree': 0.5, 'subsample': 0.7, 'learning_rate': 0.008, 'max_depth': 5, 'random_state': 22323, 'min_child_weight': 45}. Best is trial 0 with value: 0.008504432668280541.
[I 2023-09-06 05:40:36,758] Trial 1 finished with value: 0.00568106015171527 and parameters: {'lambda': 6.3486398665341, 'alpha': 2.3853762223930466, 'colsample_bytree': 0.9, 'subsample': 0.9, 'learning_rate': 0.01, 'max_depth': 10, 'random_state': 3454, 'min_child_weight': 124}. Best is trial 1 with value: 0.00568106015171527.
[I 2023-09-06 05:40:38,715] Trial 2 finished with value: 0.0038765440877504354 and parameters: {'lambda': 1.1672425985258936, 'alpha': 0.5841828994742365, 'colsample_bytree': 0.6, 'subsample': 1, 'learning_rate': 0.01, 'max

In [16]:
find_param.best_trial.params

{'lambda': 1.1672425985258936,
 'alpha': 0.5841828994742365,
 'colsample_bytree': 0.6,
 'subsample': 1,
 'learning_rate': 0.01,
 'max_depth': 9,
 'random_state': 10,
 'min_child_weight': 33}